# AI Powered Virtual Try On


### 1.1 Import the Libraries
Before we can use tools like MediaPipe to analyze images, we need to clean and organize the data. This step ensures that our data is in the correct format and ready for experiments.

In [ ]:
pip install opencv-python

In [ ]:
pip install mediapipe

In [ ]:
# Imports
import os
import cv2  # OpenCV for image processing
import numpy as np
import pandas as pd
from tqdm import tqdm
import mediapipe as mp 
import matplotlib.pyplot as plt

# Suppress warnings to keep output clean
import warnings
warnings.filterwarnings('ignore')

# Display progress bars
tqdm.pandas()

### 1.2 Load & Validate Data and Create Pairs Manually

Sometimes, the file containing image pairs (train_pairs.txt) needs manual formatting. We manually create a dataframe where each row corresponds to a pair of an image and a piece of clothing.

In [ ]:
# Sample image pairs (manually formatted from train_pairs.txt)
image_cloth_pairs = [
    ("00577_00.jpg", "00024_00.jpg"),
    ("00018_00.jpg", "04879_00.jpg"),
    ("00025_00.jpg", "06990_00.jpg"),
    ("00023_00.jpg", "04608_00.jpg"),
    ("00009_00.jpg", "07274_00.jpg"),
    ("00000_00.jpg", "08733_00.jpg")
]

In [ ]:
# Create a dataframe for pairs
data = pd.DataFrame(image_cloth_pairs, columns=["image", "cloth"])

# Show the dataframe
print("Manually created image-cloth pairs:")
print(data)


### 1.3 Validate and Organize Files
Before preprocessing, ensure that all the required files are in their respective folders. If any file is missing, this step will alert us.


In [ ]:
# Define the raw data folder paths
RAW_DATA_PATH = "/data/raw/train"
IMAGE_FOLDER = os.path.join(RAW_DATA_PATH, "image")
CLOTH_FOLDER = os.path.join(RAW_DATA_PATH, "cloth")

In [ ]:
 # Check for missing files
missing_files = []

for _, row in data.iterrows():
    image_path = os.path.join(IMAGE_FOLDER, row['image'])
    cloth_path = os.path.join(CLOTH_FOLDER, row['cloth'])

    if not os.path.exists(image_path):
        missing_files.append(image_path)
    if not os.path.exists(cloth_path):
        missing_files.append(cloth_path)

if missing_files:
    print("Missing files detected:", missing_files)
else:
    print("All files are present.")


### 2.1 Preprocess Images: Why Resize and Normalize?
To make all images uniform in size for analysis and to scale pixel values between 0 and 1 (normalization).

In [ ]:
# Directory for processed data
# PROCESSED_DATA_PATH = "./data/processed/"
# os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
IMG_SIZE = (256, 192)  # Resize dimensions (width, height)

# Function to preprocess images
def preprocess_image(image_path, output_path, img_size):
    """
    Resize and normalize an image, then save it.
    """
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load {image_path}")
        return

    # Resize the image
    img_resized = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)

    # Normalize the image (scale pixel values between 0 and 1)
    img_normalized = img_resized / 255.0

    # Save the processed image
    cv2.imwrite(output_path, (img_normalized * 255).astype(np.uint8))


In [ ]:
# Preprocess each pair
for _, row in tqdm(data.iterrows(), total=len(data)):
    img_path = os.path.join(IMAGE_FOLDER, row['image'])
    cloth_path = os.path.join(CLOTH_FOLDER, row['cloth'])

    preprocess_image(img_path, os.path.join(PROCESSED_DATA_PATH, "image", row['image']), IMG_SIZE)
    preprocess_image(cloth_path, os.path.join(PROCESSED_DATA_PATH, "cloth", row['cloth']), IMG_SIZE)

### 3.1 Visualize Processed Images  
It's always good to visually check a few processed images to ensure they look correct.

In [ ]:
# Function to plot images
def plot_images(image_paths, titles, ncols=3):
    nrows = int(np.ceil(len(image_paths) / ncols))
    plt.figure(figsize=(15, 5 * nrows))
    for i, (img_path, title) in enumerate(zip(image_paths, titles)):
        img = cv2.imread(img_path)
        plt.subplot(nrows, ncols, i + 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis('off')
    plt.show()

In [ ]:
# Example visualization
sample_images = data.sample(3)
image_paths = [os.path.join(PROCESSED_DATA_PATH, "image", img) for img in sample_images["image"]]
cloth_paths = [os.path.join(PROCESSED_DATA_PATH, "cloth", cloth) for cloth in sample_images["cloth"]]
plot_images(image_paths + cloth_paths, ["Image"] * 3 + ["Cloth"] * 3)

### 3.2 Experiment with MediaPipe
MediaPipe helps analyze pose and other features from images. This step involves loading preprocessed images and running them through MediaPipe for pose detection.

In [ ]:
# Initialize MediaPipe pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [ ]:
# Function to run MediaPipe on an image
def run_mediapipe_on_image(image_path):
    """
    Detect poses using MediaPipe and display key points.
    """
    # Read the image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # MediaPipe works with RGB images

    # Process the image
    results = pose.process(img_rgb)

    # Draw keypoints on the image
    annotated_image = img.copy()
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the image
    plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
# Example: Run MediaPipe on one image
run_mediapipe_on_image(os.path.join(PROCESSED_DATA_PATH, "image", data.iloc[0]['image']))